In [4]:
from transformers import pipeline, AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict
import evaluate
import homemade_functions as hf

model_path = "FinchW/my-yelp-sentiment-model-finetuned"
data_path = "yelp_review_full"


In [5]:
ds = load_dataset(data_path)
ds = ds.map(hf.remap_labels)


Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3, id2label=hf.id2label, label2id=hf.label2id)

pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, truncation=True) 

Device set to use cpu


In [22]:
test1 = "meal was wonderful"
test2 = "meal was terrible"


In [23]:
results = pipe(ds['test']['text'])

In [24]:
results

[{'label': 'NEGATIVE', 'score': 0.9992259740829468},
 {'label': 'NEGATIVE', 'score': 0.9993298053741455},
 {'label': 'NEGATIVE', 'score': 0.9993357062339783},
 {'label': 'NEGATIVE', 'score': 0.9986168146133423},
 {'label': 'NEGATIVE', 'score': 0.9993404746055603},
 {'label': 'NEUTRAL', 'score': 0.5176307559013367},
 {'label': 'NEGATIVE', 'score': 0.7794074416160583},
 {'label': 'POSITIVE', 'score': 0.9565002918243408},
 {'label': 'POSITIVE', 'score': 0.9952685236930847},
 {'label': 'NEUTRAL', 'score': 0.8431389927864075},
 {'label': 'NEUTRAL', 'score': 0.6193579435348511},
 {'label': 'POSITIVE', 'score': 0.5333552360534668},
 {'label': 'NEGATIVE', 'score': 0.9977641105651855},
 {'label': 'NEUTRAL', 'score': 0.6875405311584473},
 {'label': 'POSITIVE', 'score': 0.9859142303466797},
 {'label': 'POSITIVE', 'score': 0.8063162565231323},
 {'label': 'POSITIVE', 'score': 0.8560107946395874},
 {'label': 'NEGATIVE', 'score': 0.997925877571106},
 {'label': 'NEGATIVE', 'score': 0.9994169473648071}